# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [15]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,fuglafjordur,62.2440,-6.8140,9.10,95,78,6.26,FO,1731170827
1,1,blackmans bay,-43.0167,147.3167,10.07,88,99,0.45,AU,1731170829
2,2,port-aux-francais,-49.3500,70.2167,5.04,89,81,9.54,TF,1731170830
3,3,keflavik,64.0049,-22.5624,8.24,93,100,4.12,IS,1731170831
4,4,bam,29.1060,58.3570,19.78,30,0,2.20,IR,1731170832


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humidity = city_data_df["Humidity"].astype(int)
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,mount isa,-20.7333,139.5000,26.87,57,40,0.00,AU,1731170833
13,13,adamstown,-25.0660,-130.1015,22.81,88,95,8.19,PN,1731170842
16,16,ribeira grande,38.5167,-28.7000,22.10,94,40,5.66,PT,1731170845
27,27,mansourah,31.0364,31.3807,21.23,65,86,3.67,EG,1731170857
33,33,menongue,-14.6585,17.6910,23.33,59,95,0.81,AO,1731170864
...,...,...,...,...,...,...,...,...,...,...
559,559,as samawah,31.3320,45.2944,24.47,22,100,3.39,IQ,1731171466
560,560,hinatuan,8.3722,126.3342,24.41,96,90,2.05,PH,1731171467
567,567,paka,4.6374,103.4368,24.99,88,100,1.54,MY,1731171474
590,590,smyrna,33.8840,-84.5144,22.28,71,75,4.63,US,1731171500


### Step 3: Create a new DataFrame called `hotel_df`.

In [26]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[
    ['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
5,mount isa,AU,-20.7333,139.5000,57,
13,adamstown,PN,-25.0660,-130.1015,88,
16,ribeira grande,PT,38.5167,-28.7000,94,
27,mansourah,EG,31.0364,31.3807,65,
33,menongue,AO,-14.6585,17.6910,59,
...,...,...,...,...,...,...
559,as samawah,IQ,31.3320,45.2944,22,
560,hinatuan,PH,8.3722,126.3342,96,
567,paka,MY,4.6374,103.4368,88,
590,smyrna,US,33.8840,-84.5144,71,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [30]:
# Set parameters to search for a hotel
categories = "accommodation.hotel"
radius = 10000
params = {
    "categories":categories,
    "apiKey":geoapify_key   
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mount isa - nearest hotel: Ibis Styles
adamstown - nearest hotel: No hotel found
ribeira grande - nearest hotel: Hospedaria JSF
mansourah - nearest hotel: Marshall
menongue - nearest hotel: No hotel found
mandal - nearest hotel: No hotel found
eraan - nearest hotel: Trucut Travellers Inn
kinkala - nearest hotel: hotel
ponta delgada - nearest hotel: Hotel Talisman
uturoa - nearest hotel: Hawaiki Nui hotel
nova sintra - nearest hotel: Residência Ka Dencho
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
saint-leu - nearest hotel: Iloha Seaview Hôtel
necochea - nearest hotel: Cabañas Río Quequén
kalmunai - nearest hotel: Saji new food
nabire - nearest hotel: Hotel Anggrek
namibe - nearest hotel: Nelsal Pensao
puerto baquerizo moreno - nearest hotel: Dory's House
west island - nearest hotel: Cocos Village Bungalows
vredendal - nearest hotel: No hotel found
avarua - nearest hotel: Paradise Inn
bikenibeu village - nearest hotel: Otintaai Hotel
pisco - 

,City,Country,Lat,Lng,Humidity,Hotel Name
5,mount isa,AU,-20.7333,139.5000,57,Ibis Styles
13,adamstown,PN,-25.0660,-130.1015,88,No hotel found
16,ribeira grande,PT,38.5167,-28.7000,94,Hospedaria JSF
27,mansourah,EG,31.0364,31.3807,65,Marshall
33,menongue,AO,-14.6585,17.6910,59,No hotel found
...,...,...,...,...,...,...
559,as samawah,IQ,31.3320,45.2944,22,فندق قصر الغدير
560,hinatuan,PH,8.3722,126.3342,96,Lucky Cha Hometel
567,paka,MY,4.6374,103.4368,88,Paka Inn
590,smyrna,US,33.8840,-84.5144,71,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [32]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 670,
    frame_height = 500,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)